In [ ]:
import sympy as sp
thp, thd, tp, td, lp, ld, alp, r, s = sp.symbols(r'\theta_p \theta_d t_p t_d l_p l_d \alpha r s')


In [ ]:
## 1. distal actuation, no tendons on the proximal
kda = td / (ld * r)
kd1 = alp * kda
kd = 2 * (kda - kd1) * s / ld + kd1

kpa = tp / (lp * r)
kp1 = alp * kpa
kp = 2 * (kpa - kp1) * s / lp + kp1

듀얼벤딩 해석
- Distal 텐던 구동시 distal tendon에 의해 발생한 proximal angle 변화를 계산

- 동시에, proximal tendon의 길이 차분을 계산, 오프셋 부여

이후 목표 proximal tendon의 구동에 따른 proximal, distal angle의 차분을 계산

연립방정식으로 풀기

Distal tendon 구동에 의한 proximal tendon 길이 차분

힘줄의 길이 변화 $l_i - l'_i = r \kappa l_i$가 됨을 이용

In [ ]:
# proximal tendon 각도 및 길이 차분
# thp_d = sp.integrate(kd, (s, 0, lp))
# dp = thp_d * r
# dd = sp.integrate(kd, (s, lp, ld)) * r
# display(thp_d)
# display(dp)
# dp = (ld - td) * (lp / ld)
# dd = (ld - td) - dp
dp = sp.integrate(r * kd, (s, 0, lp))
dd = sp.integrate(r * kd, (s, lp, ld))
display(dp)
display(dd)

#check
display(sp.integrate(r * kd, (s, 0, ld)))
display(dp + dd)

2가지 경우 // 수정중
1. 동일방향으로 prox. tendon 추가구동시: l_p + tp가 구속조건이 됨
2. 반대방향으로 prox. tendon 추가구동시: l_p - tp가 구속조건이 됨

dist.tendon은 각 segment에서 미치는 영향력이 다름

prox seg에서는 prox tendon 구속에 의해 반지름 2r에 백본길이가 구속길이와 같아짐

dist seg에서는 prox seg에서 발생한 tendon 길이 차분에 의해 다소 변동이 발생

Case 해석
1. 동일방향
Prox. tendon만 구동했을 때 늘어난 쪽 tendon의 곡률:

$\kappa_{p,out} = \kappa_p \frac{l_p}{l_p+t_p}$

곡률반경의 닮음비를 이용하여 계산가능함

이 상태에서 디스탈을 구동

유효 백본 길이는 $l_d + t_p$가 된 상태

distal에 의한 곡률 형성: 평균곡률 먼저 구하기

$\kappa_{avg, d} = \begin{cases}
                    \frac{t_d}{2r(l_d+t_p)} (0 \le s \le l_p + t_p) \\
                    \frac{t_d}{r(l_d+t_p)}  (l_p + t_p \le s \le l_d + t_p)\\ 
                    \end{cases}$

이걸 가지고 prox 구속텐던의 곡률을 얻는다 >> 곡률 기반으로 구속안된쪽이 얼마나 수축한지를 구한다 >> 전체 길이에서 이거 빼고 디스털 구동 빼면 디스털 세그먼트 텐던 길이가 나옴 >> 디스털 길이에서 이걸 빼면 유효 디스털 TDL이 나온다

대충 $\cos(\phi_s)$ 이런식으로 일반화하면 될 듯 하다

In [ ]:
# 일반화
p_s = sp.symbols('\phi_s')
dir = sp.cos(p_s)
tdconst = lp + tp * dir
kda_pr = td / (2 * r * tdconst)
kd1_pr = alp * kda_pr
kd_pr = 2 * (kda_pr - kd1_pr) * s / tdconst + kd1_pr

kpa = tp / (lp * r)
kp1 = alp * kpa
kp = 2 * (kpa - kp1) * s / lp + kp1
kp_off = kp * (lp / tdconst)

kp_sum = kd_pr + kp_off

# proximal tendon 각도 = prox.seg 각도
pang_int = sp.integrate(kp_sum, (s, 0, tdconst))
display(pang_int)

dp = 2 * r * pang_int
prox_diff = r * pang_int # 중심축 길이 축소분

td_eff = td - dp * dir - prox_diff

# eqns
pang_eqn = sp.Eq(thp, pang_int)
dang_eqn = sp.Eq(thd, td_eff / r)
display(pang_eqn)
display(dang_eqn)

sol = sp.solve([pang_eqn, dang_eqn], (tp, td))


(l_p + t_p*cos(\phi_s))**2*(-\alpha*l_p*t_d - 2*\alpha*l_p*t_p - 2*\alpha*t_p**2*cos(\phi_s) + l_p*t_d + 2*l_p*t_p + 2*t_p**2*cos(\phi_s))/(2*l_p**3*r + 4*l_p**2*r*t_p*cos(\phi_s) + 2*l_p*r*t_p**2*cos(\phi_s)**2) + (l_p + t_p*cos(\phi_s))*(\alpha*t_d + 2*\alpha*t_p)/(2*l_p*r + 2*r*t_p*cos(\phi_s))

Eq(\theta_p, (l_p + t_p*cos(\phi_s))**2*(-\alpha*l_p*t_d - 2*\alpha*l_p*t_p - 2*\alpha*t_p**2*cos(\phi_s) + l_p*t_d + 2*l_p*t_p + 2*t_p**2*cos(\phi_s))/(2*l_p**3*r + 4*l_p**2*r*t_p*cos(\phi_s) + 2*l_p*r*t_p**2*cos(\phi_s)**2) + (l_p + t_p*cos(\phi_s))*(\alpha*t_d + 2*\alpha*t_p)/(2*l_p*r + 2*r*t_p*cos(\phi_s)))

Eq(\theta_d, (-2*r*((l_p + t_p*cos(\phi_s))**2*(-\alpha*l_p*t_d - 2*\alpha*l_p*t_p - 2*\alpha*t_p**2*cos(\phi_s) + l_p*t_d + 2*l_p*t_p + 2*t_p**2*cos(\phi_s))/(2*l_p**3*r + 4*l_p**2*r*t_p*cos(\phi_s) + 2*l_p*r*t_p**2*cos(\phi_s)**2) + (l_p + t_p*cos(\phi_s))*(\alpha*t_d + 2*\alpha*t_p)/(2*l_p*r + 2*r*t_p*cos(\phi_s)))*cos(\phi_s) - r*((l_p + t_p*cos(\phi_s))**2*(-\alpha*l_p*t_d - 2*\alpha*l_p*t_p - 2*\alpha*t_p**2*cos(\phi_s) + l_p*t_d + 2*l_p*t_p + 2*t_p**2*cos(\phi_s))/(2*l_p**3*r + 4*l_p**2*r*t_p*cos(\phi_s) + 2*l_p*r*t_p**2*cos(\phi_s)**2) + (l_p + t_p*cos(\phi_s))*(\alpha*t_d + 2*\alpha*t_p)/(2*l_p*r + 2*r*t_p*cos(\phi_s))) + t_d)/r)

TypeError: list indices must be integers or slices, not Symbol

In [ ]:
# phi_s = sp.symbols(r'\phi_s')
# tdconst = lp + sp.cos(phi_s) * tp ## 구속길이
# kda_pr = td / (2 * r * tdconst)
# kd1_pr = alp * kda_pr
# kd_pr = 2 * (kda_pr - kd1_pr) * s / (ld + sp.cos(phi_s) * td) + kd1_pr

# kpa = tp / (lp * r)
# kp1 = alp * kpa
# kp = 2 * (kpa - kp1) * s / lp + kp1
# kp_off = kp * lp / tdconst
# # proximal tendon 곡률
# k_pr_tendon = kp_off + kd_pr

# #prox. angle
# thp_int = sp.integrate(k_pr_tendon, (s, 0, tdconst))
# dp = thp_int * r * 2
# display(dp)

2*r*((l_p + t_p*cos(\phi_s))**2*(-2*\alpha*l_d*t_p - \alpha*l_p*t_d - 2*\alpha*t_d*t_p*cos(\phi_s) + 2*l_d*t_p + l_p*t_d + 2*t_d*t_p*cos(\phi_s))/(2*l_d*l_p**2*r + 2*l_d*l_p*r*t_p*cos(\phi_s) + 2*l_p**2*r*t_d*cos(\phi_s) + 2*l_p*r*t_d*t_p*cos(\phi_s)**2) + (l_p + t_p*cos(\phi_s))*(\alpha*t_d + 2*\alpha*t_p)/(2*l_p*r + 2*r*t_p*cos(\phi_s)))

In [ ]:
display(sp.sign(90 * -90))

In [ ]:
# efftp = tp + sp.Rational(1,2) * dp
# efftd = ld - 2 * lp - sp.Rational(1,2) * dp + tp

# tpeqn = efftp / r - thp
# tdeqn = efftd / r - thd

# sol = sp.solve([tpeqn, tdeqn], (tp, td))
# # Display the solution
# display(sol[tp])
# display(sol[td])